In [4]:
class Node:
    
    def __init__(self, val, nxt = None):
        self.val = val
        self.next = nxt

# Adjacency List Graph

In [3]:
#undirected graph representation
class Graph:
    
    def __init__(self, num_vertices):
        self.num_vertices = num_vertices + 1
        self.edges = [None] * (self.num_vertices)
    
    
    def add_edge(self,src,dest):
        
        new_node = Node(dest)
        new_node.next = self.edges[src]
        self.edges[src] = new_node
        
        #add other direction
        new_node = Node(src)
        new_node.next = self.edges[dest]
        self.edges[dest] = new_node
    
    def print_graph(self):
        for i in range(self.num_vertices):
            temp = self.edges[i]
            print(i, end='')
            while temp: 
                print(" -> {}".format(temp.val), end="") 
                temp = temp.next
            print(" \n")

In [ ]:
V = 5
graph = Graph(V) 

graph.add_edge(1, 2) 
graph.add_edge(1, 3) 
graph.add_edge(1, 4) 
graph.add_edge(2, 3) 
graph.add_edge(3, 4) 

graph.print_graph() 

# Simple Dictionary implementation

In [ ]:
#simple graph is represented as a dictionary with an adjacency list
graph  = {'A' : ['B', 'C'],
          'B' : ['C', 'D'],
          'C' : ['D'],
          'D' : ['C'], 
          'E' : ['F'], 
          'F' : ['C']}

In [ ]:
def find_path(graph, start, end, path=[]):
    path = path + [start]
    
    if start == end:
        return path
    
    if not graph.has_key(start):
        return None
    
    for node in graph[start]:
        if node not in path:
            newpath = find_path(graph, node, end, path)
            if newpath:
                return newpath
    return None

# Skiena Graph Converted to Python

In [1]:
'''
This graph implementation is from Skiena Algorithm Design Manual. Converted from the original C implementation
Found in Chapter 5

'''


from collections import deque 
MAXV = 1000

class SkienaNode:
    
     def __init__(self, value, nxt = None, weight = None):
            self.y = value
            self.weight = weight
            self.nxt = nxt
            

class SkienaGraph:
    
    def __init__(self, directed):
        self.num_vertices = 0
        self.directed = directed
        self.nedges = 0
        self.edges = [None]*(MAXV+1)
        self.degree = [0]*(MAXV+1)
        
    def read_from_file(self, filename):
        
        with open(filename) as f:
            line = f.readline()
            num_vert, num_edge = self.process_line(line)
            self.num_vertices, self.nedges = num_vert, num_edge
            for line in f:
                v1,v2 = self.process_line(line)
                self.insert_edge(v1,v2, self.directed)
                
    def insert_edge(self, x, y, dir_state):
        new_edge = SkienaNode(y)
        
        new_edge.nxt = self.edges[x]
        self.edges[x] = new_edge
        self.degree[x] += 1
        
        if dir_state == False:
            self.insert_edge(y,x, True)
        else:
            self.nedges += 1
            
            
    def print_graph(self):
        
        for i in range(1,self.num_vertices+1):
            print(str(i) + ": ", end='')
            neighbors = self.edges[i]
            while neighbors != None:
                print(str(neighbors.y) + ' ', end ='')
                neighbors = neighbors.nxt
            
            print()
            
            
    def process_line(self, line):
        vals = line.split(' ')
        num_vert, num_edge = [int(v) for v in vals]
        return num_vert, num_edge
    



    

# New DFS Try

In [2]:
def gfs_wrapper(g,v_start):
    
    num_nodes = g.num_vertices + 1
    visited = [False for i in range(num_nodes)]
    
    gfs(g,v_start,visited)
    
def gfs(g,v,visited):
    
    visited[v] = True
    proc_vertex(v)
    
    p = g.edges[v]
    
    while p != None:
        pval = p.y
        
        if not visited[pval]:
            visited[pval] = True
            proc_edge(v,pval)
            gfs(g,pval,visited)
        
        p = p.nxt
            

    
def proc_vertex(v):
    print(f'processing vertex {v}')

def proc_edge(s,e):
    print(f'processing edge {s} to {e}')
    

In [6]:
gtest = SkienaGraph(False)
gtest.read_from_file('./input_files/graph3.txt')
gtest.print_graph()
print()
print('graham')
gfs_wrapper(gtest,1)
print()
print('skiena')
skiena_dfs(gtest,1)

1: 3 2 
2: 3 1 
3: 1 2 

graham
processing vertex 1
processing edge 1 to 3
processing vertex 3
processing edge 3 to 2
processing vertex 2

skiena
processed vertex 1
processed edge 1 to 3
processed vertex 3
processed edge 3 to 1
processed edge 3 to 2
processed vertex 2
processed edge 2 to 3
processed edge 2 to 1


# BFS Implementation

In [ ]:
#for disconnected graph
#ensures all branches are searched
def BFS_wrapper(graph):
    processed = [False for i in range(MAXV+1)]
    discovered = [False for i in range(MAXV+1)]
    parent = [-1 for i in range(MAXV+1)]
    
    for u in range(1,MAXV+1):
        if discovered[u] == False:
            skiena_bfs_disconnected(graph, u, discovered, processed, parent)
    
def skiena_bfs_connected(graph, start):
    
    processed = [False for i in range(MAXV+1)]
    discovered = [False for i in range(MAXV+1)]
    parent = [-1 for i in range(MAXV+1)]
    
    q = deque()
    
    q.append(start)
    discovered[start] = True 
    
    while len(q) > 0:
        v = q.popleft()
        process_vertex_early(v)
        processed[v] = True
        p = graph.edges[v]
        while p != None:
            y = p.y
            if (processed[y] == False or graph.directed):
                process_edge(v,y)
            if (discovered[y] == False):
                q.append(y)
                discovered[y] = True
                parent[y] = v
            
            p = p.nxt
        process_vertex_late(v)
        
def skiena_bfs_disconnected(graph, start, discovered, processed, parent):
    
    q = deque()
    
    q.append(start)
    discovered[start] = True 
    
    while len(q) > 0:
        v = q.popleft()
        p = graph.edges[v]
        processed[v] = True
        
        if p != None:
            process_vertex_early(v)
        
        while p != None:
            y = p.y
            if (processed[y] == False or graph.directed):
                process_edge(v,y)
            if (discovered[y] == False):
                q.append(y)
                discovered[y] = True
                parent[y] = v
            
            p = p.nxt
        process_vertex_late(v)

def process_vertex_early(v):
    print(f"processed vertex {v}")

def process_edge(v,y):
    print(f"processed edge {v} to {y}")

def process_vertex_late(v):
    pass

# DFS Implementation

In [53]:
dfs_finished = False
dfs_finished2 = False

dfs_processed = [False for i in range(MAXV+1)]
dfs_discovered = [False for i in range(MAXV+1)]
dfs_parent = [-1 for i in range(MAXV+1)]
dfs_entry_time = [-1 for i in range(MAXV+1)]
dfs_exit_time = [-1 for i in range(MAXV+1)]

dfs_reachable_ancestor = [-1 for i in range(MAXV+1)]
dfs_tree_out_degree = [0 for i in range(MAXV+1)]
time = 0
  

def skiena_dfs(graph, v):
    global time
    
    if dfs_finished:
        return
    
    dfs_discovered[v] = True
    time += 1
    dfs_entry_time[v] = time
    
    
    p = graph.edges[v]
   
            
    if p != None:
        dfs_process_vertex_early(v)
        
    while p != None:
        y = p.y
        if (dfs_discovered[y] == False):
            dfs_parent[y] = v
            dfs_process_edge(v,y)
            skiena_dfs(graph,y)
        elif (not dfs_processed[y] or graph.directed):
            dfs_process_edge(v,y)
        
        if dfs_finished:
            return
        p = p.nxt
    
    dfs_process_vertex_late(v)
    
    time += 1
    dfs_exit_time[v] = time
    
    dfs_processed[v] = True
    

def dfs_process_vertex_early(v):
    print(f"processed vertex {v}")
    
def dfs_process_vertex_early2(v):
    dfs_reachable_ancestor[v] = v

def dfs_process_edge(x,y):
    print(f"processed edge {x} to {y}")
    
def dfs_process_edge2(x,y):
    cls = edge_classification(x,y)
    
    if cls == "TREE":
        dfs_tree_out_degree[x] = dfs_tree_out_degree[x] + 1
    
    if cls == "BACK" and dfs_parent[x] != y:
        if dfs_entry_time[y] < dfs_entry_time[dfs_reachable_ancestor[x]]:
            dfs_reachable_ancestor[y]

def dfs_process_vertex_late(v):
    pass



#for disconnected graph
#ensures all branches are searched
def dfs_wrapper(graph):
    
    initialize_ds()
    for u in range(1,graph.num_vertices):
        if dfs_discovered[u] == False:
            skiena_dfs(graph,u)

def initialize_ds():
    global dfs_processed
    global dfs_discovered
    global dfs_parent
    global dfs_entry_time
    global dfs_exit_time

    global dfs_reachable_ancestor
    global dfs_tree_out_degree
    global time
    
    dfs_processed = [False for i in range(MAXV+1)]
    dfs_discovered = [False for i in range(MAXV+1)]
    dfs_parent = [-1 for i in range(MAXV+1)]
    dfs_entry_time = [-1 for i in range(MAXV+1)]
    dfs_exit_time = [-1 for i in range(MAXV+1)]

    dfs_reachable_ancestor = [-1 for i in range(MAXV+1)]
    dfs_tree_out_degree = [0 for i in range(MAXV+1)]
    time = 0
    
def find_path(s,e,parents):
    if s == e or e == -1:
        print(str(s))
    else:
        find_path(s, parents[e], parents)
        print(str(e))

In [21]:
directed = True
g = SkienaGraph(True)
g.read_from_file('./input_files/graph1.txt')
#g.print_graph()

print()
#directed = True
g2 = SkienaGraph(True)
g2.read_from_file('./input_files/graph2.txt')
#g2.print_graph()
print()
#directed = True
g3 = SkienaGraph(False)
g3.read_from_file('./input_files/graph3.txt')
g3.print_graph()





1: 3 2 
2: 4 3 1 
3: 1 2 
4: 2 


In [ ]:
def uhh(inp):
    p = inp
    if inp == 2:
        return p
z = uhh(2)
print(z)

In [ ]:
skiena_bfs_connected(g,1)

In [ ]:
BFS_wrapper(g2)

In [23]:
dfs_wrapper(g3)

processed vertex 1
processed edge 1 to 3
processed vertex 3
processed edge 3 to 1
processed edge 3 to 2
processed vertex 2
processed edge 2 to 4
processed vertex 4
processed edge 4 to 2
processed edge 2 to 3
processed edge 2 to 1


# Articulation Vertex Search

In [55]:
#BRUTE FORCE SEARCH

#brute force search method where
#each vertex is temp deleted and then
#the whole graph is checked for 
#connected components to see
#that vertex created a split
# O(n(n+m))
def articulation_wrapper_brute(g):
    nv = g.num_vertices
    active = [1 for i in range(nv+1)]
    res = []
    
    for i in range(1,nv):
        dfs_processed[i] = True
        dfs_discovered[i] = True
        num_components = conn_components(g)
        if num_components > 1:
            res.append(i)
            
        initialize_ds()
    return res


def articulation_wrapper_adv(g):
    pass
        
def conn_components(g):
    
    nv = g.num_vertices
    comp_ct = 0
    for v in range(1,nv+1):
        if not dfs_processed[v]:
            comp_ct += 1
            skiena_dfs(g,v)
    
    return comp_ct




In [86]:


def process_vertex_early_adv(v):
    dfs_reachable_ancestor[v] = v
    
def process_edge_adv(x, y):
    
    cls = edge_classification(x,y)
    
    if cls == "TREE":
        dfs_tree_out_degree[x] = dfs_tree_out_degree[x] + 1
    
    if cls == "BACK" and dfs_parent[x] != y:
        if dfs_entry_time[y] < dfs_entry_time[reachable_ancestor[x]]:
            reachable_ancestor[x] = y

def edge_classification(x,y):
    if dfs_parent[y] == x:
        return "TREE"
    elif dfs_discovered[y] and not dfs_processed[y]:
        return "BACK"
    
    print(f'warning unclassified edge {x} to {y}, {dfs_discovered[1:5]}, {dfs_processed[1:5]}')
    
    
def process_vertex_late_adv(v):
    
    if dfs_parent[v] < 1:
        if dfs_tree_out_degree[v] > 1:
            print(f'root articulation vertex: {v}')
        return
    
    root = (dfs_parent[dfs_parent[v]] < 1)
    if dfs_reachable_ancestor[v] == dfs_parent[v] and not root:
        print(f'parent articulation verted: {dfs_parent[v]}')
        
    if dfs_reachable_ancestor[v] == v:
        print(f'bridge articulation vertex: {dfs_parent[v]}')
    
        if dfs_tree_out_degree[v] > 0:
            print(f'bridge articulation vertex: {v}')
            
    
    time_v = dfs_entry_time[dfs_reachable_ancestor[v]]
    time_parent = dfs_entry_time[dfs_reachable_ancestor[parent[v]]]
    
    if time_v < time_parent:
        dfs_reachable_ancestor[parent[v]] = dfs_reachable_ancestor[v]


In [52]:
#for disconnected graph
#ensures all branches are searched
def dfs_wrapper_adv(graph):
    
    initialize_ds()
    for u in range(1,graph.num_vertices):
        if dfs_discovered[u] == False:
            skiena_dfs_adv(graph,u)

def skiena_dfs_adv(graph, v):
    global time
    
    if dfs_finished:
        return
    
    dfs_discovered[v] = True
    time += 1
    dfs_entry_time[v] = time
    
    
    p = graph.edges[v]
   
            
    if p != None:
        process_vertex_early_adv(v)
        
    while p != None:
        y = p.y
        if (dfs_discovered[y] == False):
            dfs_parent[y] = v
            process_edge_adv(v,y)
            skiena_dfs_adv(graph,y)
        elif (not dfs_processed[y] or graph.directed):
            process_edge_adv(v,y)
        
        if dfs_finished:
            return
        p = p.nxt
    
    process_vertex_late_adv(v)
    
    time += 1
    dfs_exit_time[v] = time
    
    dfs_processed[v] = True

In [44]:
g_art = SkienaGraph(False)
g_art.read_from_file('./input_files/graph_artic.txt')
g_art.print_graph()

1: 2 
2: 3 1 
3: 4 2 
4: 3 


In [45]:
articulation_wrapper_brute(g_art)

processed vertex 2
processed edge 2 to 3
processed vertex 3
processed edge 3 to 4
processed vertex 4
processed edge 4 to 3
processed edge 3 to 2
processed vertex 1
processed vertex 3
processed edge 3 to 4
processed vertex 4
processed edge 4 to 3
processed vertex 1
processed edge 1 to 2
processed vertex 2
processed edge 2 to 1
processed vertex 4


[2, 3]

# Topological Sort and DAGs

In [94]:
from collections import deque

def process_vertex_late_top(v,stk):
    stk.append(v)
    
def process_edge_top(x,y):
    cls = edge_classification_DAG(x,y)
    
    if cls == "BACK":
        print("Warning cycle found, not a DAG")
        
def process_vertex_early_top(v):
    #print(f'processing vertex {v}')
    pass

def edge_classification_DAG(x,y):
    if dfs_parent[y] == x:
        return "TREE"
    if dfs_discovered[y] and not dfs_processed[y]:
        return "BACK"
    if dfs_processed[y] and (dfs_entry_time[y] > dfs_entry_time[x]):
        return "FORWARD"
    if dfs_processed[y] and (dfs_entry_time[y] < dfs_entry_time[x]):
        return "CROSS"
    
    print(f'warning unclassified edge {x} to {y}')


def topological_sort(graph):
    
    initialize_ds()
    
    numv = graph.num_vertices
    stk = deque()
    
    for u in range(1,numv):
        if dfs_discovered[u] == False:
            skiena_dfs_top(graph,u, stk)
            
    
    print_stack(stk)

def print_stack(stack):
    print("Topological Sort:")
    while len(stack) > 0:
        item = stack.pop()
        print(f"{item} ", end='')
    print()

def skiena_dfs_top(graph, v, stack):
    global time
    
    if dfs_finished:
        return
    
    dfs_discovered[v] = True
    time += 1
    dfs_entry_time[v] = time
    
    
    p = graph.edges[v]
   
            
    process_vertex_early_top(v)
        
    while p != None:
        y = p.y
        if (dfs_discovered[y] == False):
            dfs_parent[y] = v
            process_edge_top(v,y)
            skiena_dfs_top(graph,y, stack)
        elif (not dfs_processed[y] or graph.directed):
            process_edge_top(v,y)
        
        if dfs_finished:
            return
        p = p.nxt
    
    process_vertex_late_top(v, stack)
    
    time += 1
    dfs_exit_time[v] = time
    
    dfs_processed[v] = True

In [97]:
g_top = SkienaGraph(True)
g_top.read_from_file('./input_files/graph_top.txt')
g_top.print_graph()



1: 4 2 
2: 5 3 
3: 5 
4: 3 
5: 


In [98]:
topological_sort(g_top)

Topological Sort:
1 2 4 3 5 


# Tarjan's Strongly Connected Components

In [60]:
from collections import deque
MAXV = 1000

unvisited = -1
nid = 0
scc_count = 0

ids = [-1 for i in range(MAXV + 1)]
low = [0 for i in range(MAXV + 1)]
on_stack = [False for i in range(MAXV + 1)]
sccs = [-1 for i in range(MAXV + 1)]
stack = deque()

def initialize_strong_global(g):
    numv = g.num_vertices
    global ids
    global low
    global on_stack
    global stack
    global sccs
    
    ids = [-1 for i in range(numv + 1)]
    low = [0 for i in range(numv + 1)]
    on_stack = [False for i in range(numv + 1)]
    sccs = [-1 for i in range(numv + 1)]
    stack = deque()


def find_scc(g):
    numv = g.num_vertices
    
    initialize_strong_global(g)
    for i in range(1,numv + 1):
        if ids[i] == unvisited:
            strong_dfs(g, i)
            
    print(sccs)
    print(scc_count)
    

def strong_dfs(g, v):
    global nid
    global ids
    global low
    global on_stack
    global stack
    global scc_count
    
    ids[v] = low[v] = nid
    nid += 1
    stack.append(v)
    on_stack[v] = True
    
    p = g.edges[v]
    
    while p != None:
        y = p.y
        
        if ids[y] == unvisited:
            strong_dfs(g,y)
        
        if on_stack[y]:
            low[v] = min(low[y], low[v])
            
        p = p.nxt
        
    
    if ids[v] == low[v]:
        top_node = stack[-1]
        comps = []
        while len(stack) > 0:
            node = stack.pop()
            comps.append(node)
            on_stack[node] = False
            sccs[node] = scc_count
            
            if node == v:
                break
        
        print(comps)
        scc_count += 1

    
    
    
    
    
    
    
    

In [61]:
g_scc = SkienaGraph(True)
g_scc.read_from_file('./input_files/graph_scc.txt')
g_scc.print_graph()

1: 3 2 
2: 3 
3: 


In [62]:
find_scc(g_scc)

[3]
[2]
[1]
[-1, 2, 1, 0]
3


# Square of a Graph

In [41]:
import os
def square_graph(g):
    gsqr = SkienaGraph(True)
    num_vert = g.num_vertices
    
        
    with open('./input_files/gen_gsq.txt', 'w') as f:
        square_util(g, f)
        
    with open('./input_files/gen_gsq.txt', 'r') as f:  
        edge_lines = f.readlines()
        num_edges = len(edge_lines)
        edge_lines.insert(0,f'{num_vert} {num_edges}\n')
        
    with open('./input_files/gen_gsq.txt', 'w') as f: 
        f.writelines(edge_lines)
        
        
        
    gsqr.read_from_file('./input_files/gen_gsq.txt')
        
        
    
    return gsqr

def square_util(g, f):
    numv = g.num_vertices
    
    for fr in range(1, numv+1):
        to_set = dfs_lim(g,fr,2)
        #if len(to_set) > 0:
            #print(f'{fr}: {to_list}')
        for to in to_set:    
            f.write(f'{fr} {to}\n')
    
        

def dfs_lim(g, v, level):
    if level == 0:
        return set([v])
    
    level -= 1
    
    p = g.edges[v]
    neigh_hops = set()
    while p != None:
        y = p.y
        neigh_try = dfs_lim(g,y,level)
        neigh_hops = neigh_hops | neigh_try
        
        p = p.nxt 
    
    return neigh_hops
        

In [42]:
gsq_in = SkienaGraph(True)
gsq_in.read_from_file('./input_files/graph_sqr.txt')
gsq_in.print_graph()

1: 3 2 
2: 4 3 
3: 5 
4: 5 3 
5: 


In [43]:

gsq_out = square_graph(gsq_in)

gsq_out.print_graph()

1: 5 4 3 
2: 5 3 
3: 
4: 5 
5: 
